In [1]:
import torch
from torch import Tensor
from pathlib import Path
import numpy as np

/home/jsemelak/mambaforge/envs/ani-dev/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# We have a list of supported species:
#  H -> row 0
#  C -> row 1
#  N -> row 2
#  O -> row 3
#  F -> row 4
#  S -> row 5
#  Cl -> row 6

supported_species = [1, 6, 7, 8, 9, 16, 17]


s_coeffs_mus = [
                [-0.0013891633279463555, 0.052817133273316774, 0.139812833597163, 0.04197082575153015, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.3869724094534165, 1.1195726605859562, 4.597656411500856, 4.18116756869605, -0.777753667038269, 0.8991094564592526, 0.25287442792706855, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000]
]

s_coeffs_sigmas = [
                [0.014984132924379046, 0.06597323056314336, 0.05229082868259066, 0.10317217531939492, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [0.01648580936203453, 0.04360467004470585, 0.10939958699162289, 0.1452881891452822, 0.24744978852384295, 0.6757096584771374, 0.15629951845443388, 1.00000000000000000, 1.00000000000000000],      
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000]
]

p_norms_mus = [
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.04944367235325901,  0.044594583975525556, 0.04214927903340951,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000],
                [ 0.00000000000000000,  0.00000000000000000,  0.00000000000000000,  0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000, 0.00000000000000000]
]

p_norms_sigmas = [
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 0.04944367235325901,  0.044594583975525556, 0.04214927903340951,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000],
                [ 1.00000000000000000,  1.00000000000000000,  1.00000000000000000,  1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000, 1.00000000000000000]
]


data = [
        s_coeffs_mus,  
        s_coeffs_sigmas
        p_norms_mus,  
        p_norms_sigmas
    ]
    

data = torch.tensor(data, dtype=torch.float)

torch.save(data, 'normalization_library.pt')

In [4]:
data_loaded = torch.load('normalization_library.pt')

print(data_loaded)

tensor([[[-1.3892e-03,  5.2817e-02,  1.3981e-01,  4.1971e-02,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 3.8697e-01,  1.1196e+00,  4.5977e+00,  4.1812e+00, -7.7775e-01,
           8.9911e-01,  2.5287e-01,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 1.4984e-02,  6.5973e-